In [1]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm

In [2]:
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [3]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='../data', notebook_logging=True)
# Set the start and end date
start_date = '20201231'
end_date = '20201231'

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'BTC-EUR', 'LTC-USD']

In [4]:
MAX_TRAINNING_SIZE = 360
SIZE_LIMIT_REACHED = False

In [5]:
def append_ticks(tick, usd_btc, eth_btc, ltc_btc, 
                 max_r=SIZE_LIMIT_REACHED, scale_btc=1000000, scale_eth=10000, scale_ltc=10000):
    x = tick.p.btc_usd
    y = tick.p.eth_usd
    z = tick.p.ltc_usd
    usd_btc = usd_btc.append({'open': scale_btc/x.open, 'close': scale_btc/x.close, 
                      'volume': x.volume, 'datetime': tick.datetime}, ignore_index=True)
    eth_btc = eth_btc.append({'open': scale_eth * y.open/x.open, 'close': scale_eth *  y.close/x.close, 
                      'volume': y.volume, 'datetime': tick.datetime}, ignore_index=True)
    ltc_btc = ltc_btc.append({'open': scale_ltc * z.open/x.open, 'close': scale_ltc * z.close/x.close, 
                      'volume': z.volume, 'datetime': tick.datetime}, ignore_index=True)
    # Checks if size limit has been reached
    if (SIZE_LIMIT_REACHED or len(usd_btc.index) > MAX_TRAINNING_SIZE) :
        usd_btc.iloc[1:]
        eth_btc.iloc[1:]
        ltc_btc.iloc[1:]
        max_r = True
        
    return(usd_btc, eth_btc, ltc_btc, max_r)


In [6]:
def calculate_time(pddatetime, numtime, plus=True):
    if plus:
        return pddatetime.to_pydatetime() + timedelta(minutes=numtime)
    else: 
        return pddatetime.to_pydatetime() - timedelta(minutes=numtime)

# frame -- dataframe
# minute -- the number of minutes to be predicted
def predict(frame, minute=3) :
    frame = frame.set_index('datetime')
    y = frame['open'].resample('1T').mean()
    l = frame['close'].resample('1T').mean()
    v = frame['volume'].resample('1T').mean()
    merged = pd.merge(y, l, on='datetime', how="outer")
    frame = pd.merge(merged, v, on='datetime', how="outer")

    mod = sm.tsa.VARMAX(frame[['open', 'close', 'volume']], order=(1, 0), error_cov_type='diagonal')
    results = mod.fit(maxiter=50, disp=False)
    #print(results.summary().tables[1])
    newtime = calculate_time(tick.datetime, minute)
    pred = results.get_prediction(start=pd.to_datetime(frame.index[-1]), end=newtime, dynamic=False)
    pred_ci = pred.conf_int()
    #print(pred.predicted_mean)
#     print(frame.iloc[-1])
    return (pred)
# Call this function for btc prediction
#     minute -- the number of minutes to be predicted
# def predict_btc(minute) :
#     return predict(usd_btc, minute)
    
# def predict_eth(minute) :
#     return predict(eth_btc, minute)

In [7]:
start_date = '20201231'
end_date = '20201231'

products = ['BTC-USD', 'ETH-USD', 'LTC-USD']

# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)
tick = tick_gen.get_tick()

# Dataframe for history
d = {'open': [], 'close': [], 'volume': [], 'datetime': []}
usd_btc = pd.DataFrame(data=d)
eth_btc = pd.DataFrame(data=d)
ltc_btc = pd.DataFrame(data=d)

#loads initial value for testing
for i in range(0, 150):
    tick = tick_gen.get_tick()
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc)

In [8]:
ltc_btc

,open,close,volume,datetime
0,44.999252,44.923748,316.517567,2020-12-31 00:01:00+00:00
1,44.916829,44.901706,1106.098594,2020-12-31 00:02:00+00:00
2,44.887892,44.811195,1139.039085,2020-12-31 00:03:00+00:00
3,44.793977,44.594641,722.459067,2020-12-31 00:04:00+00:00
4,44.598081,44.590985,729.984393,2020-12-31 00:05:00+00:00
...,...,...,...,...
145,44.132066,44.161978,88.139252,2020-12-31 02:26:00+00:00
146,44.161971,44.132330,86.028289,2020-12-31 02:27:00+00:00
147,44.132345,44.171494,49.640441,2020-12-31 02:28:00+00:00
148,44.149580,44.133025,138.988199,2020-12-31 02:29:00+00:00


In [9]:
def get_coin_ratio(tick, holdings):
    btc = holdings.get("BTC") if "BTC" in holdings else 0
    eth = tick.p.eth_usd.close/tick.p.btc_usd.close * holdings.get("ETH") if "ETH" in holdings else 0
    ltc = tick.p.ltc_usd.close/tick.p.btc_usd.close * holdings.get("LTC") if "LTC" in holdings else 0
    usd = holdings.get("USD")/tick.p.btc_usd.close if "USD" in holdings else 0
    res = np.array([usd, eth, ltc, btc])
    return sum(res), res/sum(res)

In [10]:
def trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC):
    gap = (ideal_port - curr_port)[1:]
    gap_in_usd = gap * curr_hold_in_BTC * tick.p.btc_usd.close
    names = np.array(['ETH-USD', 'LTC-USD', 'BTC-USD'])
    names = names[np.argsort(gap_in_usd)]
    gap_in_usd = np.sort(gap_in_usd)
    # holdings = trans_tracker.get_holdings()
    
    for idx in range(len(names)):
        product = names[idx]
        change = gap_in_usd[idx]
        attr = getattr(tick.p, product.lower().replace('-', '_'))
        # curr_hold = trans_tracker.get_holdings().get(product) if product in 
        if change < 0:
            valid_vol = min(- change / attr.close, attr.volume * 0.5)
#             print(product[:3], trans_tracker.get_holdings().get(product[:3]))
            valid_vol = min(valid_vol, trans_tracker.get_holdings().get(product[:3]))
#             print(- change / attr.close, attr.volume * 0.5)
            trans_tracker.make_trade(tick, product, 'sell', valid_vol)
        elif change > 0:
            valid_amt = min(change, trans_tracker.get_holdings().get("USD"))
#             print(attr.volume)
            valid_amt = min(valid_amt, attr.volume * attr.close * 0.5)
#             print(valid_amt)
            trans_tracker.make_trade(tick, product, 'buy', valid_amt)
            
    # print(names, gap_in_usd)
    
#     btc_gap = (usd_val * b - btc) / tick.p.btc_usd.close
#     eth_gap = (usd_val * e - eth) / tick.p.eth_usd.close
#     ltc_gap = (usd_val * l - ltc) / tick.p.ltc_usd.close
    
#     if (btc_gap > 0):
#         trans_tracker.make_trade(tick, 'BTC-USD', 'sell', btc_gap / tick.p.btc_usd.close)
#     if (eth_gap > 0):
#         trans_tracker.make_trade(tick, 'ETH-USD', 'sell', eth_gap / tick.p.eth_usd.close)
#     if (ltc_gap > 0):
#         trans_tracker.make_trade(tick, 'LTC-USD', 'sell', ltc_gap / tick.p.ltc_usd.close)
        
#     if (btc_gap < 0):
#         trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(btc_gap, trans_tracker.get_holdings().get("USD")))
#     if (eth_gap < 0):
#         trans_tracker.make_trade(tick, 'ETH-USD', 'buy', min(eth_gap, trans_tracker.get_holdings().get("USD")))
#     if (ltc_gap < 0):
#         trans_tracker.make_trade(tick, 'LTC-USD', 'buy', min(ltc_gap, trans_tracker.get_holdings().get("USD")))

In [15]:
# Actual trading strategy
hr = 0
count = 0
scale_btc=1000000
scale_eth=10000
scale_ltc=10000
while tick is not None and count < 20:
    print("Time:", tick.datetime.to_pydatetime())
    print("holdings", trans_tracker.get_holdings())
#     print("Time:", tick.datetime.dt.hour)
#     print("holdings", trans_tracker.get_holdings())
    if (tick and not hr == 0 and hr != tick.datetime.to_pydatetime().hour):
        print("Time:", tick.datetime.to_pydatetime().hour)
        print("holdings", trans_tracker.get_holdings())
        hr = tick.datetime.to_pydatetime().hour
        # 10% chance of attempting to trade
#         no trade for this tick
    if (not tick.p.btc_usd.volume and not tick.p.eth_usd.volume and not tick.p.ltc_usd.volume):
        continue
    
    x_price = scale_btc / tick.p.btc_usd.close
    y_price = scale_eth * tick.p.eth_usd.close / tick.p.btc_usd.close
    z_price = scale_ltc * tick.p.ltc_usd.close / tick.p.btc_usd.close
    
    curr_hold_in_BTC, curr_port = get_coin_ratio(tick, trans_tracker.get_holdings())
    # Get predicted values
    pred_usd = predict(usd_btc, 1)
    pred_eth = predict(eth_btc, 1)
    pred_ltc = predict(ltc_btc, 1)
    
    # Find which one increases the most
    changes = np.array([(pred_usd.predicted_mean["close"][-1]-x_price)/x_price, 
               (pred_eth.predicted_mean["close"][-1]-y_price)/y_price, 
               (pred_ltc.predicted_mean["close"][-1]-z_price)/z_price, 
                       0])
    print("Changes: ", changes)

    
    # interval = (pred_usd.conf_int()["upper close"][-1] - pred_usd.conf_int()["lower close"][-1]) / ratio
#     print(count, usd_btc["close"][len(usd_btc)-1], pred_usd.conf_int()["lower close"][-1], pred_usd.conf_int()["upper close"][-1])
#     if pred_usd.predicted_mean["close"][-1] - interval <= usd_btc["close"][len(usd_btc)-1] <= pred_usd.predicted_mean["close"][-1] + interval:
#         x_acc[0] += 1
#     else:
#         x_acc[1] += 1
    
    # Obtain ideal portfolio
    ideal_port = np.zeros_like(changes)
    ideal_port[changes.argmax(0)] = 1
    
    print(count, curr_port, ideal_port)
    trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC)
    
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED)
    tick = tick_gen.get_tick()
    count += 1

Time: 2020-12-31 03:10:00+00:00
holdings {'BTC': 0.0, 'USD': 28750.984906}
Changes:  [-0.00071432 -0.00060294  0.00150936  0.        ]
0 [1. 0. 0. 0.] [0. 0. 1. 0.]
Time: 2020-12-31 03:11:00+00:00
holdings {'BTC': 0.0, 'USD': 23818.0184349014, 'LTC': 39.027400582156}
Changes:  [0.00128109 0.00080799 0.00050195 0.        ]
1 [0.82847787 0.         0.17152213 0.        ] [1. 0. 0. 0.]
Time: 2020-12-31 03:12:00+00:00
holdings {'BTC': 0.0, 'USD': 27400.095663314194, 'LTC': 10.625844632156}
Changes:  [0.00075149 0.00050333 0.0005104  0.        ]
2 [0.95325105 0.         0.04674895 0.        ] [1. 0. 0. 0.]
Time: 2020-12-31 03:13:00+00:00
holdings {'BTC': 0.0, 'USD': 28741.421235734713, 'LTC': 0.0}
Changes:  [0.00109424 0.00137024 0.00031219 0.        ]
3 [1. 0. 0. 0.] [0. 1. 0. 0.]
Time: 2020-12-31 03:14:00+00:00
holdings {'BTC': 0.0, 'USD': 16755.442449260714, 'LTC': 0.0, 'ETH': 16.193278777368}
Changes:  [0.00052545 0.00034521 0.00230534 0.        ]
4 [0.58316 0.41684 0.      0.     ] [0.

In [16]:
trans_tracker.get_holdings()

{'BTC': 0.0,
 'USD': 0.0,
 'LTC': 2.842170943040401e-14,
 'ETH': 38.41040947844947}

In [17]:
get_coin_ratio(tick, trans_tracker.get_holdings())

(0.9811614743681134,
 array([0.00000000e+00, 1.00000000e+00, 1.26767479e-16, 0.00000000e+00]))